In [ ]:
import tensorflow as tf

In [ ]:
file_url = 'https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip'

In [ ]:
zip_dir = tf.keras.utils.get_file('pizza_and_streak.zip', origin=file_url, extract=True)

In [ ]:
import pathlib

In [ ]:
path = pathlib.Path(zip_dir).parent / 'pizza_steak'

In [ ]:
train_dir = path / 'train'
validation_dir = path / 'test'

In [ ]:
train_dir

In [ ]:
train_pizza_dir = train_dir / 'pizza'
train_steak_dir = train_dir /'steak'
validation_pizza_dir = validation_dir / 'pizza'
validation_steak_dir = validation_dir / 'steak'

In [ ]:
import os

In [ ]:
total_train = len(os.listdir(train_pizza_dir)) + len(os.listdir(train_steak_dir))
total_val = len(os.listdir(validation_pizza_dir)) + len(os.listdir(validation_steak_dir))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
batch_size = 16
img_height = 224
img_width = 224

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size = batch_size,
                                                           directory = train_dir,
                                                           shuffle=True, 
                                                           target_size = (img_height, img_width),
                                                           class_mode='binary')

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size = batch_size,
                                                              directory = validation_dir,
                                                              target_size=(img_height, img_width),
                                                              class_mode='binary')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
np.random.seed(8)
tf.random.set_seed(8)

In [ ]:
import tensorflow_hub as hub

In [ ]:
MODULE_HANDLE = 'https://tfhub.dev/google/efficientnet/b0/feature-vector/1'
module = hub.load(MODULE_HANDLE)

In [ ]:
model = tf.keras.Sequential([hub.KerasLayer(MODULE_HANDLE,input_shape=(224, 224, 3)),
                             layers.Dense(500, activation='relu'),
                             layers.Dense(1, activation='sigmoid')])

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])

In [ ]:
model.fit(train_data_gen,
          steps_per_epoch = total_train // batch_size,
          epochs=5,
          validation_data=val_data_gen, 
          validation_steps=total_val // batch_size)